<a href="https://colab.research.google.com/github/AlexKressner/WS24_Supply_Chain_Optimierung/blob/main/MasterPlanning_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Master Planning

In [ ]:
! git clone https://github.com/AlexKressner/WS24_Supply_Chain_Optimierung

In [ ]:
! pip install -q pyscipopt

In [ ]:
import pandas as pd
from pyscipopt import Model, quicksum

## Daten laden

In [ ]:
folder = "WS24_Supply_Chain_Optimierung/Daten/MasterPlanning"

In [ ]:
# Kapazitäten
beschaffung_kapa = pd.read_csv(f"{folder}/Beschaffungskapazitaeten.csv", sep=";")
produktion_kapa = pd.read_csv(f"{folder}/Produktionskapazitaeten.csv", sep=";")

In [ ]:
# Kosten
produktion_kosten = pd.read_csv(f"{folder}/Produktionskosten.csv", sep=";", decimal=",")
beschaffung_kosten = pd.read_csv(f"{folder}/Beschaffungskosten.csv", sep=";", decimal=",")
lager_kosten = pd.read_csv(f"{folder}/Lagerkosten.csv", sep=";", decimal=",")
transport_beschaffung_kosten = pd.read_csv(f"{folder}/Transportkosten_Beschaffung.csv", sep=";", decimal=",")
transport_distribution_kosten = pd.read_csv(f"{folder}/Transportkosten_Distribution.csv", sep=";", decimal=",")

In [ ]:
# Direktbedarfskoeffizienten
koeffizienten = pd.read_csv(f"{folder}/Direktbedarfskoeffizienten.csv", sep=";", decimal=",")

In [ ]:
# Nachfrage
nachfrage = pd.read_csv(f"{folder}/Nachfrage.csv", sep=";")

## Indexmengen

In [ ]:
I = produktion_kapa["Werk"].unique().tolist() # Menge der Produktionsstandorte

In [ ]:
J = nachfrage["Markt"].unique().tolist() #Menge der Märkte

In [ ]:
L = beschaffung_kapa["Lieferant"].unique().tolist() # Menge der Lieferanten

In [ ]:
P = lager_kosten["Produkt"].unique().tolist() # Menge der Produkte

In [ ]:
VP = koeffizienten["Vorprodukt"].unique().tolist() # Menge der Vorprodukte

In [ ]:
FP = koeffizienten["Fertigprodukt"].unique().tolist() # Menge der Fertigprodukte

In [ ]:
A = {}
for vp in koeffizienten["Vorprodukt"].unique():
  A[vp] = koeffizienten[koeffizienten["Vorprodukt"]==vp]["Fertigprodukt"].tolist()

In [ ]:
T = beschaffung_kapa["Woche"].unique().tolist() # Menge der Planungsperioden (Wochen)

## Optimierungsmodell

In [ ]:
scip = Model()

## Entscheidungsvariablen

## Parameter

In [ ]:
# Produktionskosten
pc = produktion_kosten.set_index(["Werk","Produkt"]).to_dict("dict")["Produktionskosten"] # Kosten reguläre Produktion in €/ME
oc = produktion_kosten.set_index(["Werk"]).to_dict("dict")["Kosten_pro_Zusatzkapa"] # Kosten Zusatzkapazitäten in €/Kapazitätseinheit

In [ ]:
# Beschaffungskosten
beschaffung_kosten.set_index(["Lieferant","Produkt"], inplace=True)
bc = beschaffung_kosten.to_dict("dict")["Beschaffungskosten"] # Kosten der Beschaffung eines Produktes in €/ME

In [ ]:
# Transportkosten
transport_beschaffung_kosten.set_index(["Lieferant","Werk","Produkt"], inplace=True)
btc = transport_beschaffung_kosten.to_dict("dict")["Transportkosten"] # Kosten Transport bei Beschaffung in €/ME
transport_distribution_kosten.set_index(["Werk","Markt","Produkt"], inplace=True)
dtc = transport_distribution_kosten.to_dict("dict")["Transportkosten"] # Kosten Transport bei Distribution in €/ME

In [ ]:
# Lagerkosten
lager_kosten.set_index(["Produkt"], inplace=True)
lc = lager_kosten.to_dict("dict")["Lagerkosten"] # Kosten Lagerung in €/Woche/ME

In [ ]:
# Nachfrage
nachfrage.set_index(["Markt","Produkt","Woche"], inplace=True)
d = nachfrage.to_dict("dict")["Nachfragemenge"]

In [ ]:
# Direktbedarfskoeffiziente
koeffizienten.set_index(["Vorprodukt","Fertigprodukt"], inplace=True)
a = koeffizienten.to_dict("dict")["Direktbedarfskoeffizient"]

In [ ]:
# Kapazitäten
produktion_kapa.set_index(["Werk","Woche"], inplace=True)
pcap = produktion_kapa.to_dict("dict")["Kapaztiät"]
pcap_plus = produktion_kapa.to_dict("dict")["Max_Kapa_durch_Zusatzschichten"]
beschaffung_kapa.set_index(["Lieferant","Produkt","Woche"], inplace=True)
bcap = beschaffung_kapa.to_dict("dict")["Kapazität"]

## Zielfunktion

## Nebenbedingungen

## Berechnung Lösung

In [ ]:
scip.optimize()
print(scip.getStatus())

In [ ]:
scip.getObjVal()

In [ ]:
for t in T:
  print(f"Woche: {t}")
  for i in I:
    for p in FP:
      val = round(scip.getVal(X[i,p,t]))
    if val > 0:
      print(f"Werk {i}")
      print(f"Produktion: {val}")
    print(f"Fertigwarenbestand: {round(scip.getVal(B[i,p,t]))}")
    print("\t")
  print("\n")
